# 3. Исследование
В этой части проекта вы должны подготовить все, что используется в классах и методах, описанных выше, в Jupyter Notebook (recipes.ipynb).



### 0. Импорты

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import (train_test_split,
                                     KFold,
                                    cross_val_score,
                                    StratifiedKFold,
                                    GridSearchCV)
from sklearn.linear_model import (LogisticRegression, 
                                  LinearRegression)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (RandomForestRegressor, 
                              VotingRegressor, 
                              BaggingRegressor, 
                              StackingRegressor)
from sklearn.ensemble import (RandomForestClassifier,
                              VotingClassifier, 
                              BaggingClassifier,
                             StackingClassifier)
from sklearn.metrics import (accuracy_score, 
                             confusion_matrix, 
                             classification_report, 
                             mean_squared_error,
                            f1_score,
                            precision_score,
                            recall_score)
import math
import joblib
import requests
import json
import time

from urllib import request
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import unidecode

import warnings
warnings.filterwarnings('ignore')

## 3.1 Прогноз

### 1. Подготовка данных

* Используйте датасет из Epicurious, подготовленный HugoDarwood
* Отфильтруйте поля: удалите как можно больше столбцов, которые не относятся к названиями ингридиентов. Вы будете прогнозировать оценку или категорию оценки, используя только ингридиенты и ничего больше. Если модель обучится на других данных, которые не будут передаваться на вход, то точность прогнозов будет заведомо ниже.


In [5]:
#dataset downloaded from https://www.kaggle.com/datasets/hugodarwood/epirecipes/code?select=epi_r.csv
df_raw = pd.read_csv('data/epi_r.csv') 

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20052 entries, 0 to 20051
Columns: 680 entries, title to turkey
dtypes: float64(679), object(1)
memory usage: 104.0+ MB


Удаляем полные дубли из датасета

In [4]:
df_raw.drop_duplicates(['title', 'rating'], keep='first', inplace=True)

In [5]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18033 entries, 0 to 20051
Columns: 680 entries, title to turkey
dtypes: float64(679), object(1)
memory usage: 93.7+ MB


In [6]:
df_raw.shape

(18033, 680)

In [7]:
df = df_raw.copy()

In [8]:
list1 = ['calories', 'protein', 'fat', 'sodium',]
df.drop(list1, axis=1, inplace=True, errors='ignore')

In [9]:
list2 = [ '#cakeweek', '#wasteless', '22-minute meals', 
         '3-ingredient recipes', '30 days of groceries', 'advance prep required', 'alabama', 'alaska', 'alcoholic', 
         'anniversary', 'cookbooks', 'leftovers', 'snack', 'snack week', 'anthony bourdain', 'aperitif', 'appetizer', 
         'winter', 'wisconsin', 'wok', 'yonkers', 'yuca', 'arizona', 'windsor', 'back to school', 'backyard bbq', 
         'bastille day', 'blender', 'boil', 'bon appétit', 'bon app��tit', 'boston','braise', 'beverly hills', 
         'birthday', 'australia', 'broil', 'breakfast', 'brooklyn', 'brunch', 'tested & improved', 'texas', 'thanksgiving',
        'utah', "valentine's day", 'vegetarian', 'washington', 'washington, d.c.', 'westwood', 'tennessee', 'wedding', 
         'weelicious', 'west virginia', 'switzerland', 'california', 'camping', 'christmas', 'christmas eve',
        'georgia', 'friendsgiving','game', 'condiment/spread', 'connecticut', 'cook like a diner', 'cookbook critic', 'self', 
         'side', 'simmer', 'skewer','slow cooker', 'smoker', 'smoothie', 'snapper', 'soufflé/meringue', 'soup/stew',  
         'south carolina', 'soy free', 'spring', 'st. louis', "st. patrick's day",   'stew', 'stir-fry',  'sugar conscious', 
        'stuffing/dressing', 'summer', 'super bowl', 'suzanne goin',  'virginia','san francisco', 'sandwich', 
         'sandwich theory', 'santa monica', 'atlanta', 'bake', 'buffet', 'bulgaria', 'burrito', 'cake', 'cambridge', 'canada',
         'candy thermometer', 'casserole/gratin',  'cinco de mayo',  'cobbler/crumble', 'cocktail', 'costa mesa', 'cuba', 
         'cupcake','dairy', 'dairy free', 'dallas',   'deep-fry', 'denver', 'dessert', 'digestif', 'dinner','diwali',
         'dominican republic', 'dorie greenspan', 'double boiler','drink', 'drinks', 'easter', 'eau de vie', 'edible gift',
         'egypt', 'emeril lagasse','endive', 'engagement party', 'england', 'entertaining', 'epi + ushg',
         'epi loves the microwave', 'escarole', 'fall', 'family reunion','fat free', "father's day", 'flaming hot summer', 
         'florida', 'fontina', 'food processor','fourth of july', 'france', 'frangelico', 'frankenrecipe', 'freeze/chill', 
         'freezer food', 'frittata', 'frozen dessert',  'fry', 'germany', 'gourmet', 'graduation', 'grill', 'grill/barbecue', 
         'haiti', 'halloween', 'hamburger',  'hanukkah', 'harpercollins', 'hawaii', 'healdsburg','healthy',  'high fiber', 
         'hollywood', 'hot drink', 'house & garden', 'house cocktail', 'houston',  'ice cream machine', 'iced coffee', 
         'iced tea', 'idaho',   'illinois', 'indiana', 'iowa', 'ireland', 'israel', 'italy',  'jamaica', 'japan',  'juicer',  
         'kansas', 'kansas city', 'kentucky', 'kentucky derby', 'kid-friendly', 'kidney friendly',  'kitchen olympics', 
         'kosher', 'kosher for passover',  'labor day', 'lancaster', 'las vegas', 'london',   'long beach', 'los angeles', 
         'louisiana', 'louisville', 'low cal', 'low carb', 'low cholesterol', 'low fat', 'low sodium', 'low sugar',        
         'low/no sugar', 'lunar new year', 'lunch',  'macaroni and cheese', 'maine', 'fruit', 'vegetable', 'tropical fruit', 
        'seafood', 'nut',]
df.drop(list2, axis=1, inplace=True, errors='ignore')

In [10]:
list3 = ['mardi gras', 'margarita', 'marinade', 'marinate', 'maryland', 'massachusetts', 'mexico', 'miami',  'michigan', 
         'microwave', 'minneapolis', 'minnesota', 'mississippi', 'missouri', 'mixer',  'mortar and pestle', "mother's day", 
        'muffin',  'new hampshire',  'new jersey', 'new mexico', 'new orleans', "new year's day", "new year's eve", 
         'new york', 'no meat, no problem', 'no sugar added', 'no-cook', 'non-alcoholic','north carolina',  
         'nancy silverton', 'nebraska', 'ohio', 'oklahoma', 'oktoberfest', 'omelet', 'one-pot meal', 'oregon', 
         'organic', 'pan-fry', 'pancake',  'parade', 'paris', 'pasadena', 'passover',  'pasta maker',  'peanut free', 
         'pennsylvania', 'persian new year', 'pescatarian', 'philippines','picnic', 'pie',  'pittsburgh', 'pizza',
        'poker/game night',  'portland', 'pot pie',  'potato salad', 'potluck', 'pressure cooker', 'providence', 'purim', 
         'quick & easy', 'quick and healthy', 'ramadan', 'raw', 'rhode island', 'ramekin', 'roast',  'rosh hashanah/yom kippur',
        'seattle',  'shower', 'spain',  'steam', 'sukkot', 'tart',  'tree nut free',  'vegan', 'wheat/gluten-free', 'buffalo', 
         'chicago','chill', 'colorado', 'columbus', 'rub',  'cr��me de cacao', 'oscars', 'party', 'pastry', 'paleo', 
         'sauté', 'brownie', 'cocktail party', 'coffee grinder', 'fritter', "hors d'oeuvre", 'kwanzaa', 'pacific palisades',
        'poppy', 'quiche', 'shavuot', 'sourdough', 'vermont', 'crêpe', 'dip', 'chile','peru',]
df.drop(list3, axis=1, inplace=True, errors='ignore')

In [11]:
list4 = ['amaretto', 'apple juice', 'whiskey',  'bourbon', 'brandy', 
        'calvados', 'campari', 'chambord', 'champagne', 'chartreuse', 'coffee', 'cognac/armagnac',
         'fortified wine', 'fruit juice', 'grand marnier', 'grappa', 'jícama', 'kahlúa',
         'kirsch', 'lemon juice', 'lime juice', 'liqueur', 'marsala', 'martini', 'mezcal', 'midori',
         'orange juice', 'pomegranate juice', 'rosé', 'sake',  'sangria',
         'scotch', 'sherry',  'sparkling wine','spirit', 'spritzer','sugar snap pea','tea', 'tequila',
         'triple sec','vermouth','créme de cacao','beer','bitters', 
        ]
df.drop(list4, axis=1, inplace=True, errors='ignore')

In [12]:
list5 = ['candy','cookie', 'cookies', 'cranberry sauce', 'flat bread', 'meatball','sauce','soy sauce','spice','waffle',
         'hominy/cornmeal/masa','halibut', 'mandoline','aspen', ]
df.drop(list5, axis=1, inplace=True, errors='ignore')

In [13]:
#заменим названия
df.rename(columns = {
'milk/cream' : 'milk',
'green onion/scallion' : 'green onion', 
'butterscotch/caramel' : 'butterscotch',
'phyllo/puff pastry dough' : 'phyllo',
'jam or jelly' : 'jam',
'jalapeño' : 'jalapeno', 
'marscarpone' : 'mascarpone', 
'sweet potato/yam' : 'sweet potato'
                     }, inplace = True)

In [14]:
df.to_csv('data/dataset_clean.csv', index='title')
df.drop(['title'], axis=1, inplace=True, errors='ignore')

In [15]:
df

,rating,almond,anchovy,anise,apple,apricot,artichoke,arugula,asian pear,asparagus,...,watercress,watermelon,white wine,whole wheat,wild rice,wine,yellow squash,yogurt,zucchini,turkey
0,2.500,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,4.375,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20050,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
df.columns[0:100]

Index(['rating', 'almond', 'anchovy', 'anise', 'apple', 'apricot', 'artichoke',
       'arugula', 'asian pear', 'asparagus', 'avocado', 'bacon', 'banana',
       'barley', 'basil', 'bass', 'bean', 'beef', 'beef rib', 'beef shank',
       'beef tenderloin', 'beet', 'bell pepper', 'berry', 'biscuit',
       'blackberry', 'blue cheese', 'blueberry', 'bok choy', 'bran', 'bread',
       'breadcrumbs', 'brie', 'brine', 'brisket', 'broccoli', 'broccoli rabe',
       'brown rice', 'brussel sprout', 'bulgur', 'butter', 'buttermilk',
       'butternut squash', 'butterscotch', 'cabbage', 'cantaloupe', 'capers',
       'caraway', 'cardamom', 'carrot', 'cashew', 'cauliflower', 'caviar',
       'celery', 'chard', 'cheddar', 'cheese', 'cherry', 'chestnut', 'chicken',
       'chickpea', 'chile pepper', 'chili', 'chive', 'chocolate', 'cilantro',
       'cinnamon', 'citrus', 'clam', 'clove', 'coconut', 'cod',
       'collard greens', 'condiment', 'coriander', 'corn', 'cornmeal',
       'cottage cheese',

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18033 entries, 0 to 20051
Columns: 293 entries, rating to turkey
dtypes: float64(293)
memory usage: 40.4 MB


### Удаляем пустые рецепты (там где rating есть, а колонка с признаком удалена)

In [18]:
df_cut = df.T

In [19]:
df_cut != 0

,0,1,2,3,4,5,6,7,8,9,...,20038,20040,20043,20044,20046,20047,20048,20049,20050,20051
rating,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
almond,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
anchovy,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
anise,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
apple,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wine,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
yellow squash,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
yogurt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
zucchini,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [20]:
(df_cut != 0).any(axis=0)
#df_cut.loc[:, (df_cut != 0).any(axis=0)]
df_cut = df_cut.loc[:, (df_cut != 0).any(axis=0)]
df_cut = df_cut.T
df_cut.shape

(17942, 293)

In [21]:
y = df_cut['rating']

In [22]:
df_cut2 = df_cut.copy()
df_cut2.drop('rating', axis=1, inplace=True, errors='ignore')
X = df_cut2

In [23]:
X.columns.tolist()

['almond',
 'anchovy',
 'anise',
 'apple',
 'apricot',
 'artichoke',
 'arugula',
 'asian pear',
 'asparagus',
 'avocado',
 'bacon',
 'banana',
 'barley',
 'basil',
 'bass',
 'bean',
 'beef',
 'beef rib',
 'beef shank',
 'beef tenderloin',
 'beet',
 'bell pepper',
 'berry',
 'biscuit',
 'blackberry',
 'blue cheese',
 'blueberry',
 'bok choy',
 'bran',
 'bread',
 'breadcrumbs',
 'brie',
 'brine',
 'brisket',
 'broccoli',
 'broccoli rabe',
 'brown rice',
 'brussel sprout',
 'bulgur',
 'butter',
 'buttermilk',
 'butternut squash',
 'butterscotch',
 'cabbage',
 'cantaloupe',
 'capers',
 'caraway',
 'cardamom',
 'carrot',
 'cashew',
 'cauliflower',
 'caviar',
 'celery',
 'chard',
 'cheddar',
 'cheese',
 'cherry',
 'chestnut',
 'chicken',
 'chickpea',
 'chile pepper',
 'chili',
 'chive',
 'chocolate',
 'cilantro',
 'cinnamon',
 'citrus',
 'clam',
 'clove',
 'coconut',
 'cod',
 'collard greens',
 'condiment',
 'coriander',
 'corn',
 'cornmeal',
 'cottage cheese',
 'couscous',
 'crab',
 'cranbe

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=21, 
                                                    stratify=y)

### 2. Регрессия

* Для прогнозирования рейтинга опробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение на основе кроссвалидации и оцените RMSE на тестовой подвыборке.

### 2.1 Linear Regression

In [25]:
lr1 = LinearRegression(fit_intercept=False)
cross_val_score(lr1, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-5.552083783739233e+23

In [26]:
lr2 = LinearRegression(fit_intercept=True)
cross_val_score(lr2, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-5.98874033398359e+22

In [27]:
lr_best_reg = lr2

### 2.2 Decision Tree Regression

In [28]:
tree_params = dict(max_depth=range(10,60, 20),
                 max_features = range(10,60,20),
                 min_samples_split= [2,6,10],
                          random_state=[21])

In [29]:
tree = DecisionTreeRegressor()
grid_tree = GridSearchCV(tree, tree_params, n_jobs=-1, return_train_score = True)
grid_tree.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'max_depth': range(10, 60, 20),
                         'max_features': range(10, 60, 20),
                         'min_samples_split': [2, 6, 10],
                         'random_state': [21]},
             return_train_score=True)

In [30]:
grid_tree.best_params_

{'max_depth': 10,
 'max_features': 50,
 'min_samples_split': 10,
 'random_state': 21}

In [31]:
tree_best_reg=DecisionTreeRegressor(**grid_tree.best_params_)
cross_val_score(tree_best_reg, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-1.6204873017076082

### 2.3 Random Forest Regression

In [32]:
forest_params = dict(n_estimators=[5,10,50,100],
                 max_depth = range(10,60,20),
                 max_features = range(10,60,20),
                 random_state=[21])

In [33]:
forest=RandomForestRegressor()
grid_forest = GridSearchCV(forest, forest_params, cv=10, n_jobs=-1, return_train_score = True)
grid_forest.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': range(10, 60, 20),
                         'max_features': range(10, 60, 20),
                         'n_estimators': [5, 10, 50, 100],
                         'random_state': [21]},
             return_train_score=True)

In [34]:
grid_forest.best_params_

{'max_depth': 30, 'max_features': 10, 'n_estimators': 100, 'random_state': 21}

In [35]:
forest_best_reg = RandomForestRegressor(**grid_forest.best_params_)
cross_val_score(forest_best_reg, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-1.5670708678177263

Выберите лучшее решение на основе кроссвалидации и оцените RMSE на тестовой подвыборке.

Лучшая модель на основе кроссвалидации - Random Forest

In [36]:
forest_best_reg.fit(X_train, y_train)
y_pred = forest_best_reg.predict(X_test)
MSE = mean_squared_error(y_test, y_pred)

RMSE = math.sqrt(MSE)
print("Mean Square Error: ", MSE)
print("Root Mean Square Error: ", RMSE)

Mean Square Error:  1.5609618676495118
Root Mean Square Error:  1.24938459557076


* Попробуйте различные ансамбли и их гиперпараметры. Выберите лучшее решение на основе кроссвалидации и оцените RMSE на тестовой подвыборке.

### Voting Regressor

In [37]:
estimators = [('lr', lr_best_reg), ('tree', tree_best_reg), ('rfc', forest_best_reg)]
voting = VotingRegressor(estimators=estimators)
cross_val_score(voting, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-6.654155926651068e+21

In [38]:
voting2=VotingRegressor(estimators=estimators, weights=(2,1,2))
cross_val_score(voting2, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-9.581984534376588e+21

In [39]:
voting3=VotingRegressor(estimators=estimators, weights=(2,1,4))
cross_val_score(voting3, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-4.88876761958086e+21

In [40]:
voting4=VotingRegressor(estimators=estimators, weights=(1,2,3))
cross_val_score(voting4, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-1.663538981663755e+21

### Bagging Regressor

In [41]:
bagging_lr = BaggingRegressor(base_estimator=lr_best_reg, n_estimators=10, random_state=21, n_jobs=-1)
cross_val_score(bagging_lr, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-2.21515479031126e+22

In [42]:
bagging_lr = BaggingRegressor(base_estimator=lr_best_reg, n_estimators=20, random_state=21, n_jobs=-1)
cross_val_score(bagging_lr, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-5.819299512940826e+21

In [43]:
bagging_tree = BaggingRegressor(base_estimator=tree_best_reg, n_estimators=10, random_state=21, n_jobs=-1)
cross_val_score(bagging_tree, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-1.584904224279955

In [44]:
bagging_tree = BaggingRegressor(base_estimator=tree_best_reg, n_estimators=20, random_state=21, n_jobs=-1)
cross_val_score(bagging_tree, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-1.5817018240653442

In [45]:
bagging_forest = BaggingRegressor(base_estimator=forest_best_reg, n_estimators=10, random_state=21, n_jobs=-1)
cross_val_score(bagging_forest, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-1.5646388777700468

In [46]:
bagging_forest = BaggingRegressor(base_estimator=forest_best_reg, n_estimators=20, random_state=21, n_jobs=-1)
cross_val_score(bagging_forest, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-1.5654829351240944

### Stacking Regressor

In [47]:
estimators = [('lr', lr_best_reg), ('tree', tree_best_reg), ('rfc', forest_best_reg)]

stacking = StackingRegressor(estimators=estimators, passthrough = False,
                      final_estimator=RandomForestRegressor(),
                        cv=5, n_jobs=-1)
cross_val_score(stacking, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-1.7442798632599334

In [48]:
stacking = StackingRegressor(estimators=estimators, passthrough = True,
                      final_estimator=RandomForestRegressor(),
                        cv=7, n_jobs=-1)
cross_val_score(stacking, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-1.6756244654078027

Выберите лучшее решение на основе кроссвалидации и оцените RMSE на тестовой подвыборке.

In [49]:
essemble_best = BaggingRegressor(base_estimator=forest_best_reg, n_estimators=20, random_state=21, n_jobs=-1)

In [50]:
# лучшая модель - на тесте

essemble_best.fit(X_train, y_train)
y_pred = essemble_best.predict(X_test)
MSE = mean_squared_error(y_test, y_pred)

RMSE = math.sqrt(MSE)
print("Mean Square Error: ", MSE)
print("Root Mean Square Error: ", RMSE)

Mean Square Error:  1.5608396433835428
Root Mean Square Error:  1.2493356808254308


* Посчитайте RMSE для наивного регрессора, в котором вы просто высчитали средний рейтинг.

In [51]:
y_mean = y_test.mean()
y_pred = [y_mean for i in range(len(y_test))]

In [52]:
MSE_naive = mean_squared_error(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = math.sqrt(MSE)
print("Mean Square Error: ", MSE)
print("Root Mean Square Error: ", RMSE)

Mean Square Error:  1.6998790546498967
Root Mean Square Error:  1.3037940997910278


## 3. Классификация

* Бинаризируйте значения целевой переменной путем округления рейтингов до ближайшего целого числа. Это и будут ваши классы.
* Для прогнозирования классов попробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение на основе кроссвалидации и посчитайте accuracy на тестовой подвыборке.

In [53]:
y2 = y.round(0).astype(int)
y2

0        2
1        4
2        4
3        5
4        3
        ..
20047    3
20048    4
20049    4
20050    4
20051    4
Name: rating, Length: 17942, dtype: int32

In [54]:
X2 = X.copy()

In [55]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, 
                                                    test_size=0.2, 
                                                    random_state=21, 
                                                    stratify=y2)

* Logistic Regression

In [56]:
lr_parametrs = dict(random_state=[21], 
                    fit_intercept=[True,False])

In [57]:
clf_lr = LogisticRegression()
grid_lr = GridSearchCV(clf_lr, lr_parametrs, scoring='accuracy', n_jobs=-1)
grid_lr.fit(X2_train, y2_train)
grid_lr.best_params_

{'fit_intercept': True, 'random_state': 21}

In [58]:
grid_lr.best_score_

0.6763738793679921

In [59]:
clf_lr_best=LogisticRegression(**grid_lr.best_params_)
cross_val_score(clf_lr_best, X2_train, y2_train).mean()

0.6763738793679921

* Decision Tree

In [60]:
tree_parametrs = dict(max_depth=range(1,50,10),
                 class_weight =['balanced', None],
                 criterion = ['entropy', 'gini'],
                          random_state=[21])

In [61]:
clf_tree = DecisionTreeClassifier()
grid_tree = GridSearchCV(clf_tree, tree_parametrs, scoring='accuracy', n_jobs=-1)
grid_tree.fit(X2_train, y2_train)
grid_tree.best_params_

{'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 1,
 'random_state': 21}

In [62]:
grid_tree.best_score_

0.674562663763673

In [63]:
clf_tree_best=DecisionTreeClassifier(**grid_tree.best_params_)
cross_val_score(clf_tree_best, X2_train, y2_train).mean()

0.674562663763673

* Random Forest

In [64]:
forest_parametrs = dict(n_estimators=[5,10,50,100],
                 max_depth = range(1,50, 10),
                 max_features= range(10,60,20),
                 class_weight =['balanced', None],
                 criterion = ['entropy', 'gini'],
                          random_state=[21])

In [65]:
clf_forest= RandomForestClassifier()
grid_forest = GridSearchCV(clf_forest, forest_parametrs, scoring='accuracy', n_jobs=-1)
grid_forest.fit(X2_train, y2_train)
grid_forest.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 21,
 'max_features': 30,
 'n_estimators': 100,
 'random_state': 21}

In [66]:
grid_forest.best_score_

0.6758862929426428

In [67]:
clf_forest_best= RandomForestClassifier(**grid_forest.best_params_)
cross_val_score(clf_forest_best, X2_train, y2_train).mean()

0.6758862929426428

In [68]:
# лучшая модель - на тесте
clf_forest_best= RandomForestClassifier(**grid_forest.best_params_)
clf_forest_best.fit(X2_train, y2_train)
y2_pred = clf_forest_best.predict(X2_test)
accuracy_score(y2_test, y2_pred).mean()

0.6765115631095012

* Рассчитайте значение accuracy наивного классификатора, в котором вы для всех наблюдений прогнозируете наиболее распространенный класс. Сравните метрики.

In [69]:
# посчитаем accuracy для наивного случая
y2_pop = y2.value_counts(ascending=False).index[0] #выявили самый популярный класс
y2_naive = [y2_pop for i in range(len(y2))]
acc = accuracy_score(y2, y2_naive)
print(f'Accuracy of naiv prediction is {acc}')

Accuracy of naiv prediction is 0.6678742615093077


* Снова проведите бинаризацию целевого столбца, преобразовав теперь целые числа в классы bad (0, 1) (невкусное), so-so (2, 3) (нормальное), great (4, 5) (вкусное).
* Снова для прогнозирования классов попробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение на основе кроссвалидации и посчитайте accuracy на тестовой подвыборке.

In [70]:
y3 = y2.map({0: 'bad',
                          1: 'bad', 
                          2: 'so-so', 
                          3: 'so-so', 
                          4: 'great', 
                          5: 'great'}) 

In [71]:
X3 = X2.copy()

In [72]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, 
                                                    test_size=0.2, 
                                                    random_state=21, 
                                                    stratify=y3)

* Logistic Regression

In [73]:
lr_parametrs = dict(random_state=[21], 
                    fit_intercept=[True,False])

In [74]:
clf_lr = LogisticRegression()
grid_lr = GridSearchCV(clf_lr, lr_parametrs, scoring='accuracy', n_jobs=-1)
grid_lr.fit(X3_train, y3_train)
grid_lr.best_params_

{'fit_intercept': True, 'random_state': 21}

In [75]:
grid_lr.best_score_

0.8042920130052174

In [76]:
clf_lr_best=LogisticRegression(**grid_lr.best_params_)
cross_val_score(clf_lr_best, X3_train, y3_train).mean()

0.8042920130052174

* Decision Tree

In [77]:
tree_parametrs = dict(max_depth=range(1,50,10),
                 class_weight =['balanced', None],
                 criterion = ['entropy', 'gini'],
                          random_state=[21])

In [78]:
clf_tree = DecisionTreeClassifier()
grid_tree = GridSearchCV(clf_tree, tree_parametrs, scoring='accuracy', n_jobs=-1)
grid_tree.fit(X3_train, y3_train)
grid_tree.best_params_

{'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 1,
 'random_state': 21}

In [79]:
grid_tree.best_score_

0.8046401537907005

In [80]:
clf_tree_best=DecisionTreeClassifier(**grid_tree.best_params_)
cross_val_score(clf_tree_best, X3_train, y3_train).mean()

0.8046401537907005

* Random Forest

In [81]:
forest_parametrs = dict(n_estimators=[5,10,50,100],
                 max_depth = range(10,160, 40),
                 max_features= [10, 20, 40, 80],
                 class_weight =['balanced', None],
                 criterion = ['entropy', 'gini'],
                          random_state=[21])

In [82]:
clf_forest= RandomForestClassifier()
grid_forest = GridSearchCV(clf_forest, forest_parametrs, scoring='accuracy', n_jobs=-1)
grid_forest.fit(X3_train, y3_train)
grid_forest.best_params_

{'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 40,
 'n_estimators': 100,
 'random_state': 21}

In [83]:
grid_forest.best_score_

0.8054065586782155

In [84]:
clf_forest_best= RandomForestClassifier(**grid_forest.best_params_)
cross_val_score(clf_forest_best, X3_train, y3_train).mean()

0.8054065586782155

In [85]:
# лучшая модель - на тесте
clf_forest_best= RandomForestClassifier(**grid_forest.best_params_)
clf_forest_best.fit(X3_train, y3_train)
accuracy_score(y3_test, clf_forest_best.predict(X3_test)).mean()

0.8027305656171636

In [86]:
print(classification_report(y3_test,clf_forest_best.predict(X3_test) ))

              precision    recall  f1-score   support

         bad       0.63      0.08      0.14       334
       great       0.80      1.00      0.89      2867
       so-so       0.00      0.00      0.00       388

    accuracy                           0.80      3589
   macro avg       0.48      0.36      0.34      3589
weighted avg       0.70      0.80      0.72      3589



* Снова рассчитайте значение accuracy наивного классификатора, в котором вы для всех наблюдений прогнозируете наиболее распространенный класс. Сравните метрики.
* Что хуже: спрогнозировать плохой рейтинг, который на самом деле окажется хорошим, или спрогнозировать хороший рейтинг, который на самом деле окажется плохим? Замените метрику accuracy другой соответствующей метрикой.

In [87]:
# посчитаем accuracy для наивного случая
y3_pop = y3.value_counts(ascending=False).index[0] #выявили самый популярный класс
y3_naive = [y3_pop for i in range(len(y))]
acc = accuracy_score(y3, y3_naive)
print(f'Accuracy of naiv prediction is {acc}')

Accuracy of naiv prediction is 0.7987403856872144


Лучшая модель делает прогноз на уровне наивного

* Precision можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными, 
* recall показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.
* Очевидный недостаток пары метрик precision-recall - в том, что их две: непонятно, как ранжировать алгоритмы. Чтобы этого избежать, используют F1-метрику, которая равна среднему гармоническому precision и recall. F1-метрика будет равна 1, если и только если precision = 1 и recall = 1 (идеальный алгоритм).

In [88]:
precision_score(y3_train, clf_forest_best.predict(X3_train), average='weighted')

0.8262614631541515

In [89]:
precision_score(y3_test, clf_forest_best.predict(X3_test), average='weighted')

0.701816296641488

* Для прогнозирования классов с новой метрикой опробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение и посчитайте метрику на тестовой подвыборке.

In [90]:
lr = LogisticRegression(solver='liblinear', random_state=21).fit(X3_train, y3_train)
precision_score(y3_train, lr.predict(X3_train), average='weighted')

0.7840889230202939

In [91]:
lr = LogisticRegression(solver='liblinear', random_state=21, fit_intercept=False, penalty="l2").fit(X3_train, y3_train)
precision_score(y3_train, lr.predict(X3_train), average='weighted')

0.7125669269074747

In [92]:
svc = SVC(probability=True, random_state=21).fit(X3_train, y3_train)
precision_score(y3_train, svc.predict(X3_train), average='weighted')

0.8310812775880237

In [93]:
tree = DecisionTreeClassifier(max_depth=10, random_state=21).fit(X3_train, y3_train)
precision_score(y3_train, tree.predict(X3_train), average='weighted')

0.8143333514801544

In [94]:
tree = DecisionTreeClassifier(max_depth=60, random_state=21).fit(X3_train, y3_train)
precision_score(y3_train, tree.predict(X3_train), average='weighted')

0.9049693539762981

In [95]:
forest = RandomForestClassifier(n_estimators=50, max_depth=14, random_state=21).fit(X3_train, y3_train)
precision_score(y3_train, forest.predict(X3_train), average='weighted')

0.8309665823748758

In [96]:
forest = RandomForestClassifier(n_estimators=100, max_depth=160, random_state=21, max_features=120).fit(X3_train, y3_train)
precision_score(y3_train, forest.predict(X3_train), average='weighted')

0.9503579040571525

In [97]:
print(classification_report(y3_train, forest.predict(X3_train)))

              precision    recall  f1-score   support

         bad       0.93      0.76      0.84      1337
       great       0.95      0.99      0.97     11464
       so-so       0.97      0.80      0.88      1552

    accuracy                           0.95     14353
   macro avg       0.95      0.85      0.89     14353
weighted avg       0.95      0.95      0.95     14353



In [98]:
# лучшая модель - на тесте
clf_forest_best = RandomForestClassifier(n_estimators=100, max_depth=160, random_state=21, max_features=120).fit(X3_train, y3_train)
precision_score(y3_test, clf_forest_best.predict(X3_test), average='weighted')

0.7012695755479372

In [99]:
print(classification_report(y3_test, clf_forest_best.predict(X3_test)))

              precision    recall  f1-score   support

         bad       0.33      0.20      0.25       334
       great       0.82      0.93      0.87      2867
       so-so       0.17      0.06      0.09       388

    accuracy                           0.76      3589
   macro avg       0.44      0.39      0.40      3589
weighted avg       0.70      0.76      0.73      3589



* Попробуйте различные ансамбли и их гиперпараметры. Выберите лучшее решение и посчитайте метрику на тестовой подвыборке.

In [100]:
lr_best = LogisticRegression(solver='liblinear', random_state=21)
tree_best = DecisionTreeClassifier(max_depth=120, random_state=21)
forest_best = RandomForestClassifier(n_estimators=100, max_depth=200, random_state=21, max_features=120)

In [101]:
estimators = [('lr', lr_best), ('tree', tree_best), ('rfc', forest_best)]
voting = VotingClassifier(estimators=estimators, voting='soft').fit(X3_train, y3_train)
precision_score(y3_train, voting.predict(X3_train), average='weighted')

0.9433681914761013

In [102]:
voting = VotingClassifier(estimators=estimators, weights=(1,2,3)).fit(X3_train, y3_train)
precision_score(y3_train, voting.predict(X3_train), average='weighted')

0.9473892495193291

In [103]:
voting = VotingClassifier(estimators=estimators, weights=(2,2,4)).fit(X3_train, y3_train)
precision_score(y3_train, voting.predict(X3_train), average='weighted')

0.9473892495193291

In [104]:
bag_tree = BaggingClassifier(base_estimator=tree_best, n_estimators=20, random_state=21, n_jobs=-1)
bag_tree.fit(X3_train, y3_train)
precision_score(y3_train, bag_tree.predict(X3_train), average='weighted')

0.9330099735629963

In [105]:
bag_tree = BaggingClassifier(base_estimator=tree_best, n_estimators=10, random_state=21, n_jobs=-1)
bag_tree.fit(X3_train, y3_train)
precision_score(y3_train, bag_tree.predict(X3_train), average='weighted')

0.9257662071116254

In [106]:
bag_rf = BaggingClassifier(base_estimator=forest_best, n_estimators=20, random_state=21)
bag_rf.fit(X3_train, y3_train)
precision_score(y3_train, bag_rf.predict(X3_train), average='weighted')

0.8963039206598062

In [107]:
stacking = StackingClassifier(estimators=estimators, passthrough = False,
                      final_estimator=forest_best,
                        cv=5, n_jobs=-1).fit(X3_train, y3_train)
precision_score(y3_train, stacking.predict(X3_train), average='weighted')

0.7606762338434495

In [108]:
stacking = StackingClassifier(estimators=estimators, passthrough = True,
                      final_estimator=forest_best,
                        cv=7, n_jobs=-1).fit(X3_train, y3_train)
precision_score(y3_train, stacking.predict(X3_train), average='weighted')

0.8146384046142833

In [109]:
# лучший ансамбль на тесте
estimators = [('lr', lr_best), ('tree', tree_best), ('rfc', forest_best)]
best_voting = VotingClassifier(estimators=estimators, weights=(1,2,3)).fit(X3_train, y3_train)
precision_score(y3_test, best_voting.predict(X3_test), average='weighted')

0.7035535063554375

In [110]:
print(classification_report(y3_test, best_voting.predict(X3_test)))

              precision    recall  f1-score   support

         bad       0.34      0.22      0.26       334
       great       0.82      0.93      0.87      2867
       so-so       0.18      0.05      0.07       388

    accuracy                           0.77      3589
   macro avg       0.45      0.40      0.40      3589
weighted avg       0.70      0.77      0.73      3589



### 4. Принятие решения

* Примите решение о том, что лучше использовать: регрессионную модель или классификацию. Сохраните выбранную модель. Вы будете использовать ее в основной программе.

### лучшая модель для прогнозирования класса - классификация, random forest

In [111]:
clf_forest_best = RandomForestClassifier(n_estimators=100, max_depth=160, random_state=21, max_features=120).fit(X3_train, y3_train)
joblib.dump(clf_forest_best, "best_clf.joblib")

['best_clf.joblib']

## 3.2 Пищевая ценность
* Соберите в датафрейм всю информацию о пищевой ценности продуктов из подготовленного вами и отфильтрованного набора данных (только столбцы с продуктами). Используйте для этого следующий API.
* Конвертируйте все значения в % от суточной нормы потребления. Сохраняйте только те ингридиенты, которые находятся в этой и этой таблицах.
* Сохраните измененный датафрейм в CSV-файл, который вы будете использовать в своей основной программе.

In [112]:
# daily norms
nutrients_norms = pd.read_csv("data/nutrients_renamed.csv")
nutrients_norms

,Nutrient initial,Nutrient_rename,daily_norm
0,Vitamin A,"Vitamin A, IU",900.0
1,Vitamin C,"Vitamin C, total ascorbic acid",90.0
2,Calcium,"Calcium, Ca",1300.0
3,Iron,"Iron, Fe",18.0
4,Vitamin D,Vitamin D (D2 + D3),20.0
5,Vitamin E,Vitamin E (alpha-tocopherol),15.0
6,Vitamin K,Vitamin K (phylloquinone),120.0
7,Thiamin,Thiamin,1.2
8,Riboflavin,Riboflavin,1.3
9,Niacin,Niacin,16.0


In [113]:
list(nutrients_norms.Nutrient_rename)

['Vitamin A, IU',
 'Vitamin C, total ascorbic acid',
 'Calcium, Ca',
 'Iron, Fe',
 'Vitamin D (D2 + D3)',
 'Vitamin E (alpha-tocopherol)',
 'Vitamin K (phylloquinone)',
 'Thiamin',
 'Riboflavin',
 'Niacin',
 'Vitamin B-6',
 'Folate, total',
 'Vitamin B-12',
 'Biotin',
 'Pantothenic acid',
 'Phosphorus, P',
 'Iodine, I',
 'Magnesium, Mg',
 'Zinc, Zn',
 'Selenium, Se',
 'Copper, Cu',
 'Manganese, Mn',
 'Chromium, Cr',
 'Molybdenum, Mo',
 'Chloride, Cl',
 'Potassium, K',
 'Choline, total',
 'Total lipid (fat)',
 'Fatty acids, total saturated',
 'Cholesterol',
 'Carbohydrate, by difference',
 'Sodium, Na',
 'Fiber, total dietary',
 'Protein',
 'Added Sugars, total including NLEAs']

In [114]:
nutrients_norms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Nutrient initial  35 non-null     object 
 1   Nutrient_rename   35 non-null     object 
 2   daily_norm        35 non-null     float64
dtypes: float64(1), object(2)
memory usage: 968.0+ bytes


In [115]:
# делим исходную популяцию на сеты по 100 позиций (больше блокирует сервер)

df1 = df[df.columns[1:101]]
df2 = df[df.columns[101:201]]
df3 = df[df.columns[201:300]]

In [116]:
nutr_list = []
X_not_found = []
for ingredient in list(df1.columns):
    headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36'}
    url = f'https://api.nal.usda.gov/fdc/v1/foods/list?query={ingredient}&api_key=3zemTPOdzB9eiENqa6NzfS63MpPHHXGDc8BaifvG'
    try:
        res = requests.get(url, headers=headers) 
        time.sleep(10)
        print(ingredient,res)
        data = res.json()[0]
        dset = pd.DataFrame(data)

        for i in range(len(dset)):
            if dset['foodNutrients'].loc[i]['name'] in list(nutrients_norms.Nutrient_rename):
                nutr_list.append([ingredient, dset['foodNutrients'].loc[i]['name'],dset['foodNutrients'].loc[i]['amount'] ])

        nutr_list1 = nutr_list
    except:
        X_not_found.append(ingredient)
    
nutrients_all1 = pd.DataFrame(nutr_list1, columns=['ingredient', 'nutrient','amount'])
nutrients_all1

almond <Response [200]>
anchovy <Response [200]>
anise <Response [200]>
apple <Response [200]>
apricot <Response [200]>
artichoke <Response [200]>
arugula <Response [200]>
asian pear <Response [200]>
asparagus <Response [200]>
avocado <Response [200]>
bacon <Response [200]>
banana <Response [200]>
barley <Response [200]>
basil <Response [200]>
bass <Response [200]>
bean <Response [200]>
beef <Response [200]>
beef rib <Response [200]>
beef shank <Response [200]>
beef tenderloin <Response [200]>
beet <Response [200]>
bell pepper <Response [200]>
berry <Response [200]>
biscuit <Response [200]>
blackberry <Response [200]>
blue cheese <Response [200]>
blueberry <Response [200]>
bok choy <Response [200]>
bran <Response [200]>
bread <Response [200]>
breadcrumbs <Response [200]>
brie <Response [200]>
brine <Response [200]>
brisket <Response [200]>
broccoli <Response [200]>
broccoli rabe <Response [200]>
brown rice <Response [200]>
brussel sprout <Response [200]>
bulgur <Response [200]>
butter 

,ingredient,nutrient,amount
0,almond,Protein,21.0
1,almond,Total lipid (fat),55.5
2,almond,"Carbohydrate, by difference",18.8
3,almond,"Fiber, total dietary",10.3
4,almond,"Calcium, Ca",347.0
...,...,...,...
1573,ginger,"Carbohydrate, by difference",75.0
1574,ginger,"Fiber, total dietary",0.0
1575,ginger,"Sodium, Na",250.0
1576,ginger,Cholesterol,0.0


In [117]:
X_not_found

[]

In [118]:
nutr_list = []
for ingredient in list(df2.columns):
    headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36'}
    url = f'https://api.nal.usda.gov/fdc/v1/foods/list?query={ingredient}&api_key=3zemTPOdzB9eiENqa6NzfS63MpPHHXGDc8BaifvG'
    try:
        res = requests.get(url, headers=headers) 
        time.sleep(10)
        print(ingredient,res)
        data = res.json()[0]
        dset = pd.DataFrame(data)

        for i in range(len(dset)):
            if dset['foodNutrients'].loc[i]['name'] in list(nutrients_norms.Nutrient_rename):
                nutr_list.append([ingredient, dset['foodNutrients'].loc[i]['name'],dset['foodNutrients'].loc[i]['amount'] ])
                
        nutr_list2 = nutr_list
    except:
        X_not_found.append(ingredient)
    
nutrients_all2 = pd.DataFrame(nutr_list2, columns=['ingredient', 'nutrient','amount'])
nutrients_all2

goat cheese <Response [200]>
goose <Response [200]>
gouda <Response [200]>
grains <Response [200]>
granola <Response [200]>
grape <Response [200]>
grapefruit <Response [200]>
green bean <Response [200]>
green onion <Response [200]>
ground beef <Response [200]>
ground lamb <Response [200]>
guam <Response [200]>
guava <Response [200]>
ham <Response [200]>
hazelnut <Response [200]>
herb <Response [200]>
honey <Response [200]>
honeydew <Response [200]>
horseradish <Response [200]>
hot pepper <Response [200]>
hummus <Response [200]>
ice cream <Response [200]>
jalapeno <Response [200]>
jam <Response [200]>
jerusalem artichoke <Response [200]>
kale <Response [200]>
kiwi <Response [200]>
kumquat <Response [200]>
lamb <Response [200]>
lamb chop <Response [200]>
lamb shank <Response [200]>
lasagna <Response [200]>
leafy green <Response [200]>
leek <Response [200]>
legume <Response [200]>
lemon <Response [200]>
lemongrass <Response [200]>
lentil <Response [200]>
lettuce <Response [200]>
lima bean

,ingredient,nutrient,amount
0,goat cheese,Protein,25.000
1,goat cheese,Total lipid (fat),32.100
2,goat cheese,"Carbohydrate, by difference",0.000
3,goat cheese,"Fiber, total dietary",0.000
4,goat cheese,"Calcium, Ca",714.000
...,...,...,...
1720,poblano,Vitamin B-12,0.000
1721,poblano,"Choline, total",5.500
1722,poblano,Vitamin K (phylloquinone),7.400
1723,poblano,Cholesterol,0.000


In [119]:
X_not_found

[]

In [120]:
nutr_list = []
for ingredient in list(df3.columns):
    headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36'}
    url = f'https://api.nal.usda.gov/fdc/v1/foods/list?query={ingredient}&api_key=3zemTPOdzB9eiENqa6NzfS63MpPHHXGDc8BaifvG'
    try:
        res = requests.get(url, headers=headers) 
        time.sleep(10)
        print(ingredient,res)
        data = res.json()[0]
        dset = pd.DataFrame(data)

        for i in range(len(dset)):
            if dset['foodNutrients'].loc[i]['name'] in list(nutrients_norms.Nutrient_rename):
                nutr_list.append([ingredient, dset['foodNutrients'].loc[i]['name'],dset['foodNutrients'].loc[i]['amount'] ])
                
        nutr_list3 = nutr_list
    except:
        X_not_found.append(ingredient)  
    
nutrients_all3 = pd.DataFrame(nutr_list3, columns=['ingredient', 'nutrient','amount'])
nutrients_all3

pomegranate <Response [200]>
pork <Response [200]>
pork chop <Response [200]>
pork rib <Response [200]>
pork tenderloin <Response [200]>
port <Response [200]>
potato <Response [200]>
poultry <Response [200]>
poultry sausage <Response [200]>
prosciutto <Response [200]>
prune <Response [200]>
pumpkin <Response [200]>
punch <Response [200]>
quail <Response [200]>
quince <Response [200]>
quinoa <Response [200]>
rabbit <Response [200]>
rack of lamb <Response [200]>
radicchio <Response [200]>
radish <Response [200]>
raisin <Response [200]>
raspberry <Response [200]>
red wine <Response [200]>
rhubarb <Response [200]>
rice <Response [200]>
ricotta <Response [200]>
root vegetable <Response [200]>
rosemary <Response [200]>
rum <Response [200]>
rutabaga <Response [200]>
rye <Response [200]>
saffron <Response [200]>
sage <Response [200]>
salad <Response [200]>
salad dressing <Response [200]>
salmon <Response [200]>
salsa <Response [200]>
sardine <Response [200]>
sausage <Response [200]>
scallop <R

,ingredient,nutrient,amount
0,pomegranate,Protein,0.00
1,pomegranate,Total lipid (fat),0.00
2,pomegranate,"Carbohydrate, by difference",12.10
3,pomegranate,"Fiber, total dietary",0.00
4,pomegranate,"Calcium, Ca",8.00
...,...,...,...
1606,turkey,"Sodium, Na",521.00
1607,turkey,"Vitamin A, IU",530.00
1608,turkey,"Vitamin C, total ascorbic acid",0.00
1609,turkey,Cholesterol,34.00


In [121]:
X_not_found

[]

In [122]:
df_nutr1 = pd.concat([nutrients_all1, nutrients_all2], ignore_index=True)
df_nutr = pd.concat([df_nutr1, nutrients_all3], ignore_index=True)
df_nutr

,ingredient,nutrient,amount
0,almond,Protein,21.00
1,almond,Total lipid (fat),55.50
2,almond,"Carbohydrate, by difference",18.80
3,almond,"Fiber, total dietary",10.30
4,almond,"Calcium, Ca",347.00
...,...,...,...
4909,turkey,"Sodium, Na",521.00
4910,turkey,"Vitamin A, IU",530.00
4911,turkey,"Vitamin C, total ascorbic acid",0.00
4912,turkey,Cholesterol,34.00


In [123]:
df_fin = pd.pivot_table(df_nutr, index='ingredient', columns='nutrient', values = 'amount')
df_fin.fillna(0, inplace=True)
df_fin

nutrient,Biotin,"Calcium, Ca","Carbohydrate, by difference",Cholesterol,"Choline, total","Copper, Cu","Fatty acids, total saturated","Fiber, total dietary","Folate, total","Iron, Fe",...,Thiamin,Total lipid (fat),"Vitamin A, IU",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid",Vitamin D (D2 + D3),Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),"Zinc, Zn"
ingredient,,,,,,,,,,,,,,,,,,,,,
almond,0.0,347.0,18.80,0.0,52.1,0.934,6.550,10.3,53.0,3.49,...,0.041,55.50,0.0,0.00,0.103,0.0,0.0,24.20,0.0,3.29
anchovy,0.0,232.0,0.00,85.0,85.0,0.339,2.200,0.0,13.0,4.63,...,0.078,9.71,0.0,0.88,0.203,0.0,1.7,3.33,12.1,2.44
anise,0.0,646.0,50.00,0.0,0.0,0.910,0.586,14.6,10.0,37.00,...,0.340,15.90,311.0,0.00,0.650,21.0,0.0,0.00,0.0,5.30
apple,0.0,0.0,14.30,0.0,0.0,0.000,0.000,3.2,0.0,0.23,...,0.000,0.65,65.0,0.00,0.000,3.1,0.0,0.00,0.0,0.00
apricot,0.0,0.0,13.50,0.0,0.0,0.000,0.000,0.0,0.0,0.00,...,0.000,0.00,368.0,0.00,0.000,17.6,0.0,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wild rice,0.0,0.0,77.80,0.0,0.0,0.000,0.000,4.4,0.0,1.60,...,0.000,1.11,0.0,0.00,0.444,0.0,0.0,0.00,0.0,3.33
wine,0.0,5.0,5.00,0.0,0.0,0.009,0.000,0.0,0.0,0.10,...,0.000,0.00,0.0,0.00,0.000,0.0,0.0,0.00,0.0,0.02
yellow squash,0.0,20.0,4.08,0.0,0.0,0.000,0.000,2.0,0.0,0.37,...,0.000,0.00,306.0,0.00,0.000,18.4,0.0,0.00,0.0,0.00


In [124]:
df_fin.columns

Index(['Biotin', 'Calcium, Ca', 'Carbohydrate, by difference', 'Cholesterol',
       'Choline, total', 'Copper, Cu', 'Fatty acids, total saturated',
       'Fiber, total dietary', 'Folate, total', 'Iron, Fe', 'Magnesium, Mg',
       'Manganese, Mn', 'Molybdenum, Mo', 'Niacin', 'Pantothenic acid',
       'Phosphorus, P', 'Potassium, K', 'Protein', 'Riboflavin',
       'Selenium, Se', 'Sodium, Na', 'Thiamin', 'Total lipid (fat)',
       'Vitamin A, IU', 'Vitamin B-12', 'Vitamin B-6',
       'Vitamin C, total ascorbic acid', 'Vitamin D (D2 + D3)',
       'Vitamin E (alpha-tocopherol)', 'Vitamin K (phylloquinone)',
       'Zinc, Zn'],
      dtype='object', name='nutrient')

In [125]:
# daily norms
norms = nutrients_norms.copy()

In [126]:
norms.set_index('Nutrient_rename', inplace=True)

In [127]:
df_nutr_norms = df_fin.copy() #датасет с пищевой ценностью продуктов в % от дневной нормы

In [128]:
for col in df_fin.columns:
    df_nutr_norms[col] = df_nutr_norms[col]/norms.loc[col, "daily_norm"]

In [129]:
df_nutr_norms

nutrient,Biotin,"Calcium, Ca","Carbohydrate, by difference",Cholesterol,"Choline, total","Copper, Cu","Fatty acids, total saturated","Fiber, total dietary","Folate, total","Iron, Fe",...,Thiamin,Total lipid (fat),"Vitamin A, IU",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid",Vitamin D (D2 + D3),Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),"Zinc, Zn"
ingredient,,,,,,,,,,,,,,,,,,,,,
almond,0.0,0.266923,0.068364,0.000000,0.094727,1.037778,0.3275,0.367857,0.1325,0.193889,...,0.034167,0.711538,0.000000,0.000000,0.060588,0.000000,0.000,1.613333,0.000000,0.299091
anchovy,0.0,0.178462,0.000000,0.283333,0.154545,0.376667,0.1100,0.000000,0.0325,0.257222,...,0.065000,0.124487,0.000000,0.366667,0.119412,0.000000,0.085,0.222000,0.100833,0.221818
anise,0.0,0.496923,0.181818,0.000000,0.000000,1.011111,0.0293,0.521429,0.0250,2.055556,...,0.283333,0.203846,0.345556,0.000000,0.382353,0.233333,0.000,0.000000,0.000000,0.481818
apple,0.0,0.000000,0.052000,0.000000,0.000000,0.000000,0.0000,0.114286,0.0000,0.012778,...,0.000000,0.008333,0.072222,0.000000,0.000000,0.034444,0.000,0.000000,0.000000,0.000000
apricot,0.0,0.000000,0.049091,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,...,0.000000,0.000000,0.408889,0.000000,0.000000,0.195556,0.000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wild rice,0.0,0.000000,0.282909,0.000000,0.000000,0.000000,0.0000,0.157143,0.0000,0.088889,...,0.000000,0.014231,0.000000,0.000000,0.261176,0.000000,0.000,0.000000,0.000000,0.302727
wine,0.0,0.003846,0.018182,0.000000,0.000000,0.010000,0.0000,0.000000,0.0000,0.005556,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.001818
yellow squash,0.0,0.015385,0.014836,0.000000,0.000000,0.000000,0.0000,0.071429,0.0000,0.020556,...,0.000000,0.000000,0.340000,0.000000,0.000000,0.204444,0.000,0.000000,0.000000,0.000000


In [130]:
df_nutr_norms.to_csv('data/daily_norms.csv')

In [131]:
pd.read_csv('data/daily_norms.csv')

,ingredient,Biotin,"Calcium, Ca","Carbohydrate, by difference",Cholesterol,"Choline, total","Copper, Cu","Fatty acids, total saturated","Fiber, total dietary","Folate, total",...,Thiamin,Total lipid (fat),"Vitamin A, IU",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid",Vitamin D (D2 + D3),Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),"Zinc, Zn"
0,almond,0.0,0.266923,0.068364,0.000000,0.094727,1.037778,0.3275,0.367857,0.1325,...,0.034167,0.711538,0.000000,0.000000,0.060588,0.000000,0.000,1.613333,0.000000,0.299091
1,anchovy,0.0,0.178462,0.000000,0.283333,0.154545,0.376667,0.1100,0.000000,0.0325,...,0.065000,0.124487,0.000000,0.366667,0.119412,0.000000,0.085,0.222000,0.100833,0.221818
2,anise,0.0,0.496923,0.181818,0.000000,0.000000,1.011111,0.0293,0.521429,0.0250,...,0.283333,0.203846,0.345556,0.000000,0.382353,0.233333,0.000,0.000000,0.000000,0.481818
3,apple,0.0,0.000000,0.052000,0.000000,0.000000,0.000000,0.0000,0.114286,0.0000,...,0.000000,0.008333,0.072222,0.000000,0.000000,0.034444,0.000,0.000000,0.000000,0.000000
4,apricot,0.0,0.000000,0.049091,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,...,0.000000,0.000000,0.408889,0.000000,0.000000,0.195556,0.000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,wild rice,0.0,0.000000,0.282909,0.000000,0.000000,0.000000,0.0000,0.157143,0.0000,...,0.000000,0.014231,0.000000,0.000000,0.261176,0.000000,0.000,0.000000,0.000000,0.302727
284,wine,0.0,0.003846,0.018182,0.000000,0.000000,0.010000,0.0000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.001818
285,yellow squash,0.0,0.015385,0.014836,0.000000,0.000000,0.000000,0.0000,0.071429,0.0000,...,0.000000,0.000000,0.340000,0.000000,0.000000,0.204444,0.000,0.000000,0.000000,0.000000
286,yogurt,0.0,0.090769,0.057818,0.040000,0.000000,0.000000,0.0880,0.000000,0.0000,...,0.000000,0.037692,0.065556,0.000000,0.000000,0.007778,0.000,0.000000,0.000000,0.000000


### 3.3. Похожие рецепты
* Для каждого рецепта из набора данных найдите ссылку на сайте epicurious.com и подробную информацию о нем (если сделать это не удалось, найдите для данного рецепта похожую ссылку в Интернете).
* Сохраните новый датафрейм в CSV-файл, который вы будете использовать в своей основной программе.

In [132]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Firefox/3.0.7 Chrome/104.0.5112.102'}

def get_all_recipes_epi(title): 
    title = title.rstrip() #удалим пробелы в конце
    title = unidecode.unidecode(title) # remove accents (normalize) unicode string
    title = title.replace('"','')
    title = title.replace('’','')
    title = title.replace(' ', '%20') #заменим пробелы внутри на %20
    
    base_url = 'http://www.epicurious.com'
    search_url_str = 'search'
    url = f'{base_url}/{search_url_str}/{title}?content=recipe'

    try:
        soup = BeautifulSoup(request.urlopen(
            request.Request(url, headers=HEADERS)).read(), "html.parser")

        recipe_link_items = soup.select('div.results-group article.recipe-content-card a.view-complete-item') #check this line
        recipe_links = [r['href'] for r in recipe_link_items]

        return [(url + r) for r in recipe_links]
    except (HTTPError, URLError):
        return []

### попробовать мультипроцессинг
https://ru.stackoverflow.com/questions/1070824/%d0%9a%d0%b0%d0%ba-%d1%83%d1%81%d0%ba%d0%be%d1%80%d0%b8%d1%82%d1%8c-%d0%bf%d0%b0%d1%80%d1%81%d0%b8%d0%bd%d0%b3-beautifulsoup

In [133]:
#проверка на 1 позиции
title = 'Lentil, Apple, and Turkey Wrap'
get_all_recipes_epi(title)[0]

'http://www.epicurious.com/recipes/food/views/lentil-apple-and-turkey-wrap-239173'

In [134]:
titles = list(df_raw['title'])
t1 =titles[0:100] #тест на первых 100 позициях

dict_recipes = {}
not_found = []

for title in t1:
    try:
        res = get_all_recipes_epi(title=title)
        dict_recipes[title] = res[0]
    except:
        not_found.append(title)
        dict_recipes[title] = "It's a secret recipe! Only Yandex.Eda khows how to cook it! https://eda.yandex.ru/"
        print(f"{title} is not found")

Colin Perry’s Sorghum and Apple Sticky Pudding  is not found


In [136]:
len(dict_recipes)

100

In [137]:
len(not_found)

1

In [138]:
# на всей популяции
titles = set(df_raw['title']) #берем только уникальные title

dict_recipes = {}
not_found = []

for title in titles:
    try:
        res = get_all_recipes_epi(title=title)
        dict_recipes[title] = res[0]
    except:
        not_found.append(title)
        dict_recipes[title] = "It's a secret recipe! Only Yandex.Eda khows how to cook it! https://eda.yandex.ru/"
        print(f"{title} is not found")

Tomato and Cucumber Salad with Pita Bread and Za'atar  is not found
Tom Colicchio's Herb-Butter Turkey  is not found
Nellie & Joe's Key Lime Pie  is not found
Queen's Peg  is not found
Chile-Marinated Pork With Vietnamese Brussels Sprouts  is not found
Bobby's Dry-Rubbed Rib-Eye Steaks with Mesa Barbecue Sauce  is not found
Aunt Enza's Overstewed Green Beans  is not found
Junior's Russian Dressing  is not found
Chris Lilly's Flank Steak and Shiitake Yakitori  is not found
Paula Wolfert's Seven-Day Preserved Lemons  is not found
Julia's and Jacques's Deglazing Sauce for Roast Chicken  is not found
Queen Elizabeth's Favorite Quiche  is not found
Mule's Hind Leg  is not found
Hershey's "Perfectly Chocolate" Chocolate Cake  is not found
Irene's Beer Stew  is not found
Pimm's Cup  is not found
Morel and Asparagus Sauté D'aoust  is not found
Portuguese Fisherman's Stew  is not found
Jacques's Savory-Stuffed Roast Chicken  is not found
Grandma Ethel's Brisket with Tzimmes  is not found
Fronte

Crepes with Caramel Sauce and Toasted Pecans  is not found
Pasta e Fagioli With Escarole  is not found
Grilled Pimiento Cheese and Fried Green Tomato Sandwich (GPC)  is not found
Kid-Friendly Turkey Burgers  is not found
Perciatelli with Tuna, Capers and Tomatoes  is not found
Punch à la Noix de Coco  is not found
Crispy Chicken Stew With Lemon, Artichokes, Capers, and Olives  is not found
Asian Salmon Burgers with Pickled Cucumber on Pumpernickel  is not found
Grilled Lemon-Herb Poussins  is not found
Steak Au Poivre  is not found
Passover Chremslach  is not found
Grilled Lemon-and-Gin-Marinated Chicken and Onions  is not found
Farmers' Market Quinoa Salad  is not found
Zeke's Tyropitas  is not found
Smoked Turkey and Cheese Spread  is not found
Dr. Lee's Red Wine Chicken Stew  is not found
Matzo Balls in Southeast Asian Broth  is not found
Brined Cucumber and Radish Salad  is not found
Artichoke Olive Dip with Fennel Crudites  is not found
Sliced Filet Mignon with Fava Beans, Radishe

Berry Berry Sundaes  is not found
Citrus Cooler  is not found
Meatloaf  is not found
Shaved Fennel, Artichoke, and Parmesan Salad  is not found
Balsamic Truffles  is not found
Oyster Pan Roast with Pepper Croutons  is not found
Bushranger  is not found
Chocolate Chestnut Ice Cream  is not found
Vanilla Bean Apple Crisp  is not found
Apple Cheese Quick Bread  is not found
Roasted Squash, Red Pepper, and Jack Cheese Quesadillas with Chipotle Lime Sour Cream Dip  is not found
Summer Chef's Salad with Grilled Pork, Chicken, and Chimichurri Ranch Dressing  is not found
Spicy Beef Curry  is not found
Butter Braised Oysters on Greens  is not found
Double Chocolate-Ginger Shortbread  is not found
Butterscotch Budino with Whipped Crème Fraîche  is not found
Roasted Squash with Date Relish and Pumpkin Seeds  is not found
Koffie Van Brunt  is not found
Kale and Mushrooms with Creamy Polenta  is not found
Poached Scrod with Herbed Vinaigrette  is not found
Brook Trout Müllerin  is not found
Crab J

Angel-Hair Pasta with Sauteed Squid  is not found
Papaya and Red Onion Salsa  is not found
Scandinavian Sugar-Browned Potatoes  is not found
Butternut Squash Vegducken With Mushroom-Cranberry Stuffing  is not found
Wild Mushroom and Bell Pepper Sauté  is not found
Tiny Corn Muffins  is not found
Balsamic-Glazed Salmon with Spinach, Olives, and Golden Raisins  is not found
Black Cod with Olives and Potatoes in Parchment  is not found
Cabbage and White Bean Soup with Sausage  is not found
Ratatouille Sandwich  is not found
Creamed Spinach Deluxe  is not found
Open-Face Chicken Burgers with Basil Mayonnaise  is not found
Spicy Roasted Vegetable Soup with Toasted Tortillas  is not found
Chicken Tagine with Couscous  is not found
Spicy Tomato Barbecue Sauce  is not found
Spicy Noodles with Ginger and Fresh Vegetables  is not found
Avocado and Tangerine Salad with Jalapeño Vinaigrette  is not found
"Fried" Chicken  is not found
Three-Day, Twice-Cooked Pork Roast with Fried-Herb Salsa Verde  

Triple-Ginger Cookies  is not found
Brown Rice and Chicken Stir-Fry with Edamame and Walnuts  is not found
Pumpkin and Cookie-Butter Sheet Cake With Toasted Meringue  is not found
Frisée Salad with Poached Eggs  is not found
Watermelon with Fennel Salt  is not found
Watercress, Pear and Walnut Salad with Poppy Seed Dressing  is not found
Sesame Spinach with Ginger and Garlic  is not found
Cheese, Herb, and Sun-Dried Tomato Phyllo Rolls  is not found
Hazelnut Meringues with Lemon Cream and Blood Orange Syrup  is not found
Lemon-Caper Braised Halibut  is not found
Quick Beef Burgundy  is not found
Marmalade Chicken  is not found
Chocolate-Orange Sorbet  is not found
Moroccan Garbanzo Bean and Feta Pitas  is not found
Venison with Mushroom-Wine Sauce  is not found
Ramen with Smoked Turkey Broth  is not found
Gnocchi with Tomato, Basil, and Olives  is not found
Spiced Blush Wine Compote  is not found
Grilled Corn Tabbouleh in Tomato Cups  is not found
Crudites and Grilled Sausage with Swee

...

Herbed Yellow Pepper Scrambled Eggs with Chive Sour Cream on Brioche  is not found
Orange-Almond Shortcakes with Cranberry Parfait  is not found
Lemon Macaroon Pie  is not found
One-of-Each Soup  is not found
Gumbo Z'herbes with Red Beans  is not found
Fudgy Orange-Zucchini Cake with Orange Glaze  is not found
Mexican Hot Chocolate II  is not found
Asian Steak and Noodle Salad  is not found
Roast Chicken with Pesto and Potatoes  is not found
Texican Rice  is not found
Salted Roast Turkey with Chipotle Glaze and Caramelized-Onion Gravy  is not found
Chicken Long Rice  is not found
Onion and Barley Soup with Swiss Cheese Flan  is not found
3-Ingredient Shakshuka  is not found
Honey Custard in Phyllo Pastry  is not found
Simple Smoked Beef Short Ribs  is not found
Pumpkin Bread Pudding with Cranberry Caramel Sauce  is not found
Gingered Lemon Almond Squares  is not found
Tangerine Chutney  is not found
Classic Gazpacho  is not found
White Bean and Vegetable Soup  is not found
Mango in Gin

Spelt Spaghetti, Vine Tomatoes & Baked Ricotta  is not found
Raspberry Orange Shake  is not found
Great Gazpacho  is not found
Jalapeño Popper Fritters with Tomato Salsa  is not found
Watermelon, Lime, and Tequila  is not found
Shrimp, Chicken, and Andouille Gumbo  is not found
Mac and Two Cheeses with Caramelized Shallots  is not found
Royal Fizz  is not found
Chicken Sauté with Guava Sauce  is not found
Chestnut-Caramel Ice Cream  is not found
Molasses Corn Bread  is not found
Apple-Cream Cheese Torte  is not found
Shakshuka a la Doktor Shakshuka  is not found
Long Bean Salad  is not found
Baklava  is not found
Chopped Fried-Fish Tacos (Tacos de salpicón de pescado)  is not found
Vegetable Platter with Cannellini "Hummus"  is not found
Spicy Tamarind Skirt Steak  is not found
White Wine and Peach Sangria  is not found
Lime Granita with Candied Mint Leaves and Crème Fraîche  is not found
Caramelized Banana Slices with Toasted Pecans and Frozen Yogurt  is not found
Mixed Vegetable Reli

Pork Scaloppine with Sun-Dried Tomatoes and Rosemary  is not found
Twice-Baked Potatoes with Corned Beef and Cabbage  is not found
Southern Sausage Soufflé  is not found
Caramelized Onion and Goat Cheese Tarts  is not found
Orange-Ginger Charoset  is not found
Breakfast Taco Hobo Packs With Black Beans, Zucchini, and Corn  is not found
Creamy Apple Horseradish Sauce  is not found
Summer Strawberry Jam  is not found
Herbed Roast Leg of Lamb with Roasted Onions and Potatoes  is not found
Hannah's Peanut Butter and White Chocolate Candies  is not found
Pink Grapefruit Tart with Edamame Ice Cream and Black Sesame Seeds  is not found
Peach or Nectarine Chutney  is not found
Cottage Cheese Rugelach with Walnuts  is not found
Frozen Peach and Amaretti Souffles  is not found
Gingered Nectarine Cobbler  is not found
Champagne Framboise  is not found
Creamed Spinach with Golden Breadcrumb Topping  is not found
Smoked Salmon Dip  is not found
Baked Eggs with Spinach  is not found
Lamb Chops with 

Apricot and Amaretti Crostata  is not found
Apricot-Cream Cheese Tartlets in Macadamia Crust  is not found
Mocha Brownies  is not found
Vegetable-and Ricotta- Stuffed Collard Rolls with Tomato Sauce  is not found
Broiled Portobello Topped with Creamy Scrambled Eggs  is not found
Mai Tai II  is not found
Macadamia Nut Pie  is not found
Pasta with Pesto, Broccoli, and Potatoes  is not found
Smoky Pumpkin, Spelt, Pomegranate, and Feta Salad  is not found
Celery Bisque with Stilton Toasts  is not found
Cedar-Planked Char with Wood-Grilled Onions  is not found
Game Hens with Madeira and Wild Mushrooms  is not found
Broccoli Potato Soup with Parmesan Croutons  is not found
Chocolate Sour Cream Frosting  is not found
To Toast and Skin Hazelnuts  is not found
Chocolate Macadamia Nut Sauce  is not found
Chilled Pea Broth with Lemon Cream  is not found
Pea Shoot and Spinach Salad with Bacon and Shiitakes  is not found
Butternut Squash and Corn Chowder  is not found
Poached Pear Sundaes  is not f

Cinnamon-Walnut Coffee Cake  is not found
Beef Tenderloin Steaks on Potato Galettes with Mustard Sauce  is not found
Marsala and Mascarpone Mousse with Pound Cake and Berries  is not found
Moroccan-Style Cornish Game Hens  is not found
Bbq Onion Steaks with Honey-Mustard Sauce  is not found
Stuffed Turkey with Lemon, Oregano, and Red Onions  is not found
Roasted Tomato-Chipotle Salsa  is not found
Applejack Rabbit  is not found
Grapefruit and White Beets with Yogurt and Tarragon  is not found
Sloppy Joes  is not found
Frozen Praline Souffles  is not found
Rice Pilaf with Lamb, Carrots, and Raisins  is not found
Radish and Chive Salad  is not found
Ice Cream Bombe  is not found
Anchoiade  is not found
Mashed Potatoes with Leeks and Sour Cream  is not found
Sausage-Cheese Balls  is not found
Platter of Roasted Shellfish with Trio of Sauces  is not found
Fillet of Beef, Arugula, and Artichoke Crostini  is not found
Caramel-Pistachio Torte with Halvah and Dark Chocolate  is not found
Grill

In [141]:
data = pd.DataFrame(dict_recipes, index=[0]).T
data.reset_index(inplace=True)
data.rename(columns={'index': 'title', 0: 'url'}, inplace=True)

In [142]:
data.to_csv('data/all_recipes.csv')

Соберем сводный файл: title - rating - url

In [144]:
recipes = df_raw[['title','rating']]
recipes.drop_duplicates(['title'], keep='first', inplace=True)


In [149]:
data = pd.read_csv('data/all_recipes.csv')

In [151]:
all_recipes = data.merge(recipes, how='left', left_on = 'title', right_on = 'title')

all_recipes.drop('Unnamed: 0', axis=1, inplace=True)

In [152]:
all_recipes.to_csv('data/recipes_info.csv')